In [1]:
.open incidents.db
--.load /Users/alex/projects/sqlite-jiff/dist/debug/jiff0
.load uv:sqlite-jiff
attach database 'backlog.db' as backlog;

Opened database at incidents.db

Loaded 'sqlite-jiff' with uv from /Users/alex/.cache/uv/archive-v0/d6A5MYIS90Hhn7pVdSMKK/lib/python3.13/site-packages/sqlite_jiff/jiff0.dylib

attach database 'backlog.db' as backlog;

In [2]:
drop table if exists calls;
create table calls(
  uid integer primary key,
  agency text,
  address text,
  cross_street text,
  call_at text,
  incident_type text,
  alarm_level text,
  no_units integer,
  units text,
  longitude real,
  latitude real,
  key_map text,
  combined_response text
);

drop table if exists checkpoint;
create table checkpoint(cursor timestamp);
create index checkpoint_cursor on checkpoint(cursor);

drop table if exists calls;

create table calls(
  uid integer primary key,
  agency text,
  address text,
  cross_street text,
  call_at text,
  incident_type text,
  alarm_level text,
  no_units integer,
  units text,
  longitude real,
  latitude real,
  key_map text,
  combined_response text
);

drop table if exists checkpoint;

create table checkpoint(cursor timestamp);

create index checkpoint_cursor on checkpoint(cursor);

In [3]:
select 
  min(time),
  max(time),
  jiff_until(
    jiff_datetime_strptime('%Y-%m-%d %H:%M:%S %z', min(time)), 
    jiff_datetime_strptime('%Y-%m-%d %H:%M:%S %z', max(time))
  ) as spread
from backlog.version;

min(time),max(time),spread
2025-05-20 05:40:01 +0000,2025-05-31 17:09:49 +0000,"11 days, 11 hours, 29 minutes, 48 seconds"


In [4]:
with paired as (
  select rowid, *, lead(time, 1) over (order by rowid) as next_time
  from backlog.version
)
  select 
  rowid, 
  time,
  next_time,
  iif(next_time is not null, 
    jiff_until(
      jiff_datetime_strptime('%Y-%m-%d %H:%M:%S %z', time), 
      jiff_datetime_strptime('%Y-%m-%d %H:%M:%S %z', next_time)
    )
  ) as duration
from paired
limit 10;

rowid,time,next_time,duration
1,2025-05-31 17:09:49 +0000,2025-05-31 16:53:51 +0000,"15 minutes, 58 seconds ago"
2,2025-05-31 16:53:51 +0000,2025-05-31 16:42:26 +0000,"11 minutes, 25 seconds ago"
3,2025-05-31 16:42:26 +0000,2025-05-31 16:29:51 +0000,"12 minutes, 35 seconds ago"
4,2025-05-31 16:29:51 +0000,2025-05-31 16:10:23 +0000,"19 minutes, 28 seconds ago"
5,2025-05-31 16:10:23 +0000,2025-05-31 15:52:30 +0000,"17 minutes, 53 seconds ago"
6,2025-05-31 15:52:30 +0000,2025-05-31 15:38:30 +0000,14 minutes ago
7,2025-05-31 15:38:30 +0000,2025-05-31 15:26:33 +0000,"11 minutes, 57 seconds ago"
8,2025-05-31 15:26:33 +0000,2025-05-31 15:10:02 +0000,"16 minutes, 31 seconds ago"
9,2025-05-31 15:10:02 +0000,2025-05-31 14:52:25 +0000,"17 minutes, 37 seconds ago"
10,2025-05-31 14:52:25 +0000,2025-05-31 14:39:13 +0000,"13 minutes, 12 seconds ago"


In [5]:
begin;
.param set noop 1

insert or replace into temp.sqlite_parameters(key, value) values
  (
    'cursor', 
    iif(
      (select count(*) from checkpoint) = 0,
      (select min(time) from backlog.version),
      (select max(cursor) from checkpoint)
  )
);

insert or ignore into calls
  select 
    value ->> '$.attributes.UID'               as uid,
    value ->> '$.attributes.Agency'            as agency,
    value ->> '$.attributes.Address'           as address,
    value ->> '$.attributes.CrossStreet'       as cross_street,
    jiff_timestamp_from_ms(
      value ->> '$.attributes.CALL_TIME'
    ) as call_at,
    value ->> '$.attributes.IncidentType'      as incident_type,
    value ->> '$.attributes.ALARM_LEVEL'       as alarm_level,
    value ->> '$.attributes.NO_UNITS'          as no_units,
    value ->> '$.attributes.Units'             as units,
    value ->> '$.attributes.LONGITUDE'         as longitude,
    value ->> '$.attributes.LATITUDE'          as latitude,
    value ->> '$.attributes.KeyMap'            as key_map,
    value ->> '$.attributes.CombinedResponse'  as combined_response
  from json_each(
    --readfile('getIncidentsGit/data.json'),
    (
      select data_json 
      from backlog.version 
      where time > :cursor
      order by time asc
      limit 1
    ),
    '$.features'
  );
  select :cursor, changes();
  insert into checkpoint(cursor) values (
    (
       select time
      from backlog.version 
      where time > :cursor
      order by time asc
      limit 1
    )
  );
commit;

begin;

set parameter : noop

insert or replace into temp.sqlite_parameters(key, value) values
  (
    'cursor', 
    iif(
      (select count(*) from checkpoint) = 0,
      (select min(time) from backlog.version),
      (select max(cursor) from checkpoint)
  )
);

insert or ignore into calls
  select 
    value ->> '$.attributes.UID'               as uid,
    value ->> '$.attributes.Agency'            as agency,
    value ->> '$.attributes.Address'           as address,
    value ->> '$.attributes.CrossStreet'       as cross_street,
    jiff_timestamp_from_ms(
      value ->> '$.attributes.CALL_TIME'
    ) as call_at,
    value ->> '$.attributes.IncidentType'      as incident_type,
    value ->> '$.attributes.ALARM_LEVEL'       as alarm_level,
    value ->> '$.attributes.NO_UNITS'          as no_units,
    value ->> '$.attributes.Units'             as units,
    value ->> '$.attributes.LONGITUDE'         as longitude,
    value ->> '$.attributes.LATITUDE'          as latitude,
    value ->> '$.attributes.KeyMap'            as key_map,
    value ->> '$.attributes.CombinedResponse'  as combined_response
  from json_each(
    --readfile('getIncidentsGit/data.json'),
    (
      select data_json 
      from backlog.version 
      where time > :cursor
      order by time asc
      limit 1
    ),
    '$.features'
  );

:cursor,changes()
2025-05-20 05:40:01 +0000,49


insert into checkpoint(cursor) values (
    (
       select time
      from backlog.version 
      where time > :cursor
      order by time asc
      limit 1
    )
  );

commit;